# Quant Research Intern Test — Intraday ATM Straddle 

---

#### Objective

Design a **Python pipeline** that takes the provided **two months of minute-by-minute options data and spot/index data** (02-01-2025 to 28-02-2025) , **shorts an ATM CE+PE straddle at 10:30 AM** each trading day, manages **combined intraday stop/target**, and produces a **trade sheet and metrics**.

The rest of this notebook contains an outline of the pipeline with function placeholders. Complete those functions, adhere to the trading rules below, and produce clean outputs. FAQ and definitions appear at the end of this notebook for any doubts about the terminology used below.

 There are 4 additional csv files attached. **NIFTY_option_data.csv** is the 2 months option data, **NIFTY_spot_data.csv** is the 2 months spot/index data, **SamplePerformanceStats.csv** is a sample output of performance report for reference and **SampleTradeReport.csv** is a sample output for the trading report for your reference.

#### **A more detailed explanation of what is to be done is explained right before every task. Below is an overview explanation of the task to be implemented.**

#### 1) Trading Rules (Exact Specification)

1.	Sell an ATM straddle at 10:30 am by simultaneously selling both an ATM call option and an ATM put option (**CE_Strike = PE_Strike = ATM_Strike**) with the same weekly expiry date and strike price.
2.	Calculate the 30% stop loss based on the combined premium collected from selling the straddle.
3.	Monitor the position intraday and exit all legs of the trade together if either of the following conditions is met:
-	The 30% stop loss is triggered.
-	The target of 80% is achieved (i.e., straddle premium reduces to 20% of entry value).
4.	If neither condition is met, square off the position at 3:20 pm on the same day (end of intraday session).
5.	Generate a trade report in the same format as the sample report provided with the assignment. Include details from the sample report and any other relevant information necessary for performance analysis. Trades in the attached report are for reference only, and there is no need to match them.
6.	Calculate returns.
7.	Generate performance statistics in the same format as the sample performance stats sheet provided.


#### 2) Data (What to Load and How)

##### Data horizon
- Load the **2 months options data** from the attached options CSV, and the **index (spot) data** CSV.
-In this task we will only deal with close prices.

##### Options CSV (schema & example)

NIFTY_option_data.csv — 1-minute OHLC for options with columns: DateTime (IST), ExpiryDate, StrikePrice, Type (CE|PE), Open, High, Low, Close, Ticker

**Raw columns (as provided):**
| Date       | Time  | Open   | High   | Low    | Close  | OI   | Volume | ExpiryDate | StrikePrice | Type | Ticker                           |
|------------|-------|--------|--------|--------|--------|------|--------|------------|-------------|------|-----------------------------------|
| 02-01-2025 | 09:17 | 320.05 | 326.95 | 317.35 | 326.95 | 1800 | 130650 | 16-01-2025 | 23800       | CE   | 2025010209:17NIFTY25011623800CE |

**Columns to load & standardize:**
- `Date`, `Time`, `Close`, `ExpiryDate`, `StrikePrice`, `Type`

##### Spot CSV (schema & example)

NIFTY_spot.csv — 1-minute spot with columns: DateTime (IST), Open, High, Low, Close

**Raw columns (as provided):**
| Date       | Time  | Open   | High   | Low    | Close   | OI | Volume | ExpiryDate | StrikePrice | Type | Ticker                 |
|------------|-------|--------|--------|--------|---------|----|--------|------------|-------------|------|------------------------|
| 02-01-2025 | 09:07 | 23783  | 23783  | 23783  | 23783   | 0  | 0      |            | 0           | IDX  | 2025010209:07NIFTY     |

**Columns to load & standardize:**
- `Date`, `Time`, `Close`

##### When working with the data, allow a tolerance of +1 minute 

---

#### 3) Expiry Processing (How to Map Days to Expiries)

Build an **expiry calendar**
- Expiry of an option is the day it expires. For options there are multiple expiries- nearest, next, far. For Nifty the expiry we will conider is Thursday for this task.
- Derive dates of **unique weekly expiries** for the date range given.
- Create an **expiry matrix** that lists neighbor expiries per row: **Old, I, II** (previous, current/nearest, next).

| Old        | I          | II         |
|------------|------------|------------|
|            | 2025-01-02 | 2025-01-09 |
| 2025-01-02 | 2025-01-09 | 2025-01-16 |
| 2025-01-09 | 2025-01-16 | 2025-01-23 |

- Using this expiry matrix, for every trading day you will assign expiries from here.

---

#### 4) Deliverables (What You Must Submit)

1. **`Trade_Sheet.csv`**(with the following columns) :  
   `Date, En_Date, Max_Ex_Date, Expiry, Spot_En_Price, Atm_Strike, CE_Strike, PE_Strike, Entry_Straddle, CE_Entry, PE_Entry, CE_Exit, PE_Exit, Exit_reason, Total_Returns, pnl`.
where En_date= Entry time (10:30), Max_Ex_Date= Max exit time (3:20), Spot_En_Price= Spot price at entry, ATM_strike= strike price at entry,  Entry_Straddle= total premium at entry, CE_Entry= Entry price of call, PE_Entry= Entry price of put, PE_Exit and CE_Exit are exit prices of the call and put, Exit_Reason= reason for exit (SL, Profit point, EOD). 
Refer to sample attached an idea of what the output should look like, but the values or column names might differ.

2.`tradesheet_report` output, Refer to sample attached for the output format, but the values will differ.

3. **Notebook** with **completed functions**, clean comments, docstrings, and a brief **Assumptions & Notes** section (e.g. any issues faced or comments).

---

#### 5) Evaluation Criteria

- **Correctness** (expiry mapping, ATM selection, combined SL/TP, EOD handling)  
- **Code quality** (readability, modularity, maintainability)  
- **Efficiency** (chunked reads, minimal copies, vectorization where practical)  
- **Robustness** (timestamp tolerances, nearest-strike fallback, ffill/bfill, graceful skips)  
- **Error handling** (clear exceptions, helpful messages)  
- **Documentation** (clean docstrings, comments, assumptions)  
- **Completeness** (all required features & outputs delivered)

---

#### 6) What to Hand In (Explicit List)

- The **completed notebook** (with all functions implemented and comments).  
- **`Trade_Sheet.csv'**.  
- The **'Performance report.csv'**  
- Any **helper scripts** you created (optional).



## Main Coding Task begins here

Task 1 — Loading processed options + spot data
Implement `load_processed_data(path_opt, path_spot)` to:
- read options CSV and spot CSV
- Standardize and load the following columns: `DateTime`, `Expiry`, `Type`, `Strike`, `Close`

**Candidate task:** Fill the function below. After running it you should have two DataFrames: 'options_df' and 'spot_df'

In [ ]:
def load_processed_data(path_opt, path_spot):
    """ Load processed options and spot CSVs.
Returns
-------
data : pd.DataFrame
Options intraday with DateTime index and expected columns.
"""
# --- implement ---
# 1) load options CSV into `options_df`
# 2) keep only useful columns and cast types
# 3) set DateTime index and sort
# 4) load spot into `spot_df'set DateTime index
raise NotImplementedError

Task 2 — Build Expiry DataFrame
#
Create a helper `build_expiry_matrix(options_df)` which derives unique expiry dates and builds a small dataframe with
- Old (previous expiry), I (this expiry), II (next expiry)
- used for nearest-week logic

-Expected output :

| Old        | I          | II         |
|------------|------------|------------|
| NaT        | 2025-01-02 | 2025-01-09 |
| 2025-01-02 | 2025-01-09 | 2025-01-16 |
| 2025-01-09 | 2025-01-16 | 2025-01-23 |




In [ ]:
def build_expiry_matrix(options_df):
    """Return a DataFrame that lists expiry dates and their neighbors.


The input `options_df` must have an `Expiry` datetime-like column.
"""
# --- implement ---
raise NotImplementedError

 Task 3 — Create Tradesheet skeleton
#
 Implement `create_tradesheet(options_df, spot_df, df_expiry)` such that:
 - builds a DataFrame of trading dates
 - creates an `En_Date` (entry datetime) for each date (e.g. 10:30:00) and `Max_Ex_Date` (e.g. 15:20:00)
 - assigns expiry using nearest weekly expiry after entry logic
 - returns df_ts with required columns-  Date, En_Date, Max_Ex_Date, Expiry
#


In [ ]:
def assign_expiry_to_tradesheet(df_ts, df_expiry):
    """Assign nearest expiry >= En_Date for each trade row in df_ts."""
# --- implement ---
raise NotImplementedError

In [ ]:

def create_tradesheet(options_df, spot_df, df_expiry, entry_time_str='10:30:00', exit_time_str='15:20:00'):
    """Create per-day tradesheet.


Returns df_ts with columns ['Date', 'En_Date', 'Max_Ex_Date', 'Expiry'] plus placeholders.
"""
# --- implement ---
raise NotImplementedError

Task 4 — Assign ATM strike and Spot price at En_Date
#
Implement `assign_atm_and_spot(df_ts, options_df,)` which:
- for each `En_Date` in df_ts, find the spot Close at that timestamp (close of 10:29)
- compute ATM strike (follow rounding convention: round to nearest 50)
 - add `Spot_En_Price`, `Atm_Strike` columns.
#
**Candidate requirement:** if exact timestamp missing, pick nearest earlier bar within 1 minute.

In [ ]:
def assign_atm_and_spot(df_ts, spot_df, round_to=50):
# --- implement ---
raise NotImplementedError

Task 5 — Intraday Option Series (Close-only)

For each trade in `df_ts`, build a minute series only using `Close`from the option data for the chosen **CE** and **PE** strikes between `En_Date` and `Max_Ex_Date`.  
Compute `Close_Straddle = Close_CE + Close_PE`

Function `collect_intraday_option_series(df_ts, options_data)`

**Inputs**
- `df_ts`: must have `En_Date`, `Max_Ex_Date`, `Expiry`, `CE_Strike`, `PE_Strike`.
- `options_data`: DatetimeIndex, cols at least `['Expiry','Type','Strike','Close']`.

**Steps**
1. Slice window: `win = options_df.loc[En_Date:Max_Ex_Date]` and filter `win['Expiry'] == row['Expiry']`.
3. Extract close prices for exact `CE_Strike` & `PE_Strike`
4. Build two small DFs (CE/PE) with **only `Close`**.
5. **Inner-join** on time.
6. Add column `Close_Straddle = Close_CE + Close_PE`.
7. Return the DataFrame indexed by DateTime with columns: `Close_CE`, `Close_PE`, `Close_Straddle`.  

In [ ]:
def collect_intraday_option_series(df_ts, options_df):
# --- implement ---
raise NotImplementedError

Task 6- `get_intraday_option_data(df_ts, options_df)`

**Purpose:**  
Iterate over each trade row in `df_ts`, call `collect_intraday_option_series(df_ts.loc[i], options_df)`, and return a dictionary mapping every row in df_ts -> the minute-by-minute time series between En_Date and Max_Ex_Date for that row’s assigned expiry and selected CE/PE strike.  
If a day errors or yields no data, store an **empty DataFrame** for that key.
The output should look something like this:

| DateTime           | Close_CE | Close_PE | Close_Straddle |
|--------------------|---------:|---------:|---------------:|
| 2025-01-02 10:30:00|    88.85 |   104.90 |         193.75 |
| 2025-01-02 10:31:00|    90.95 |   105.35 |         196.30 |
| 2025-01-02 10:32:00|    90.50 |   105.00 |         195.50 |
| 2025-01-02 10:33:00|    89.90 |   105.35 |         195.25 |



In [ ]:
def get_intraday_option_data(df_ts, options_df):
# --- implement ---
raise NotImplementedError

Task 7 — Track Trades & Exit Logic (Close-only)

**Input**
- `df_ts`: tradesheet (one row per day) with `En_Date`, `Max_Ex_Date`, assigned strikes/expiry.
- `result_intraday_dict`: `{i: DataFrame}` for each trade index `i`, with columns  
  `Close_CE`, `Close_PE`, and `Close_Straddle = Close_CE + Close_PE`.

**Do**
1. **Entry**  
   - `CE_Entry = first Close_CE`, `PE_Entry = first Close_PE`  
   - `Entry_Straddle = CE_Entry + PE_Entry`  
   -  Record entry prices at 10:30 close price

2. **Thresholds**  
   - `SL_level = 1.30 * Entry_Straddle` (combined stop)  
   - `TP_level = 0.20 * Entry_Straddle` (combined target)  

3. **Exit (scan forward, first condition wins)**  
   - If `Close_Straddle ≥ SL_level` → exit both legs here.  
   - Else if `Close_Straddle ≤ TP_level` → exit both legs here.   
   - If none → **EOD exit** at 3:20 close price.

4. **Record**
   - `CE_Exit`, `PE_Exit` (from exit row), `Exit_Straddle`, `Exit_Tick`  
   - Record`Exit_reason` as SL if stop hit, TP if target point reached, or EOD is exit is at EOD. 

5. **Returns (short legs)**  
   - `Ce_Short_Atm_Return = (CE_Entry - CE_Exit) / CE_Entry`  
   - `Pe_Short_Atm_Return = (PE_Entry - PE_Exit) / PE_Entry`  
   - `Total_Returns = Ce_Short_Atm_Return + Pe_Short_Atm_Return`

Return an updated df_ts (or a new DataFrame) with at least:
`Date, En_Date, Max_Ex_Date, Expiry, Spot_En_Price, Atm_Strike, CE_Strike, PE_Strike, Entry_Straddle, CE_Entry, PE_Entry, CE_Exit, PE_Exit, Exit_reason, Total_Returns
 

In [ ]:
def track_trades(df_ts, result_intraday_dict):

    
raise NotImplementedError

Task 8 — Compute PnL from `df_ts` (1 NIFTY lot) and Export CSV

- Use **only `df_ts`** columns (`CE_Entry`, `PE_Entry`, `CE_Exit`, `PE_Exit`) to compute PnL.
- Assume working with **1 lot**.
- expot the resulting table to a CSV file called Trade_Sheet.csv.

In [ ]:
def calculate_pnl():
# --- implement ---
raise NotImplementedError

 Task 9 — Metrics & Reporting
#
 Implement `tradesheet_report(df)` which computes and returns a CSV of metrics  (Definitions are given at the end of the notebook)
 1. Total Trades
 2. Profitable Trades
 3. Losing Trades
 4. Hit Ratio 
 5. Avg Return Per Trade
 6. Avg Profit Per Trade 
 7. Avg Loss Per Trade 
 8. Max Profit Per Trade 
 9. Max Loss Per Trade 
 10. Total Return
 11. Max Drawdown
 

 

In [ ]:
def tradesheet_report(df):
# --- implement (can copy/adapt original code) ---
raise NotImplementedError

### FAQs (Common Questions)

**What is an ATM straddle?**  
Selling both an **ATM CE** and **ATM PE** with the **same strike & expiry**. “ATM” (At-The-Money) means the strike closest to the current spot/futures at entry.

**How is the combined stop loss calculated?**  
On the **combined straddle premium** collected at entry. If `Entry_Straddle = P`, the stop-loss (SL) triggers when **Straddle ≥ 1.30 × P** (+30%).

**What is the combined target?**  
If the straddle premium falls to **≤ 0.20 × P** (i.e., 80% reduction), exit **both legs** and book profit.

**What if neither target nor SL hits?**  
Exit **both legs at 3:20 PM** (end of intraday session).


**What if I cannot load full data due to RAM?**  
Inform us and work with smalled dataset like **only 1 month**. If still constrained, work with 2 weeks dataset.

**What if there are some tasks I cannot complete?**
It is completely understandable, and in that case complete the tasks you can and submit it.

---

#### Definitions (Finance Terms Used)

- **ATM (At-The-Money):** Option whose **strike** is closest to the current **spot** (index) level.  
- **Straddle (Short ATM Straddle):** Strategy of **selling** an ATM **Call (CE)** and an ATM **Put (PE)** with the **same expiry and strike**.  
- **CE / PE:** **Call European** and **Put European** options (here simply “call” and “put”).  
- **Strike (Strike Price):** The fixed price at which the option can be exercised.  
- **Expiry (Expiration Date):** The date on which the option contract expires.  
- **Weekly Expiry:** Options that expire weekly (e.g., every Thursday for many index options), as opposed to monthly.  
- **Spot (Index Spot):** The real-time (or near real-time) price of the underlying index; used to define ATM.  
- **LTP (Last Traded Price):** The most recent trade price of the instrument.  
- **OI (Open Interest):** The total number of **outstanding** (open) option contracts.  
- **PCR (Put-Call Ratio):** Typically **Put OI / Call OI** (can also be volume-based) — a sentiment indicator.  
- **Stop-Loss (SL):** A predefined exit rule to cap loss; here, the **combined straddle value** reaching **+30%** from entry.  
- **Target (TP):** A predefined take-profit; here, the **combined straddle value** falling to **20%** of entry (i.e., **80% decay**).  
- **Entry_Straddle:** The **sum of CE and PE prices** at the first tick after the entry time (10:30).  
- **Intraday:** Trading within the **same trading day**; all positions are closed by session end (3:20 PM here).  
- **Lot Size:** The **contract multiplier** (e.g., 15) to convert option price change into cash PnL.  
- **Position Sizing:** Deciding **how many lots** to trade based on capital and risk (e.g., `int(20000 / ((CE+PE)*lot))`, cap at 3).  
- **PnL (Profit and Loss):** Realized gain or loss; here: `position_size × Total_Returns × lot_size`.  
- **Total_Returns (short legs):** Sum of **CE return + PE return**, where each is `(Entry − Exit) / Entry`.  
- **Equity Curve:** Plot of **cumulative PnL (or account value)** over time.  
- **Drawdown (DD):** Peak-to-trough decline of the equity curve; **Max DD** is the worst such drop.  
- **Expectancy:** Average **profit per trade** (in currency or %).  
- **Expectancy Ratio:** Expectancy relative to risk or average loss; higher implies better risk-adjusted edge.  
- **Profit Factor:** `Total gross profit / Total gross loss`; >1 indicates profitability.  
- **Win Ratio (Win Rate):** Percentage of trades with **positive PnL**.  
- **High/Low (bar fields):** The maximum/minimum traded price within a time bar (e.g., 1-minute candle).  
- **Forward/Backward Fill (ffill/bfill):** Fill small missing values by propagating last known/next known value.


### Good luck! Fill in the functions and run the notebook top to bottom. When done, export `Trade_Sheet.csv` and a short markdown cell describing any edge-cases you handled.